In [50]:
# imports 
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from shapely.geometry import LineString, Point

from processing import *

## Read the shapefile width and depth data
A simple near-global database of bankfull widths and depths (along with confidence intervals) was developed based on hydraulic geometry equations and the HydroSHEDS hydrography data set. The bankfull width estimates were evaluated with widths derived from Landsat imagery for reaches of nine major rivers, showing errors ranging from 8 to 62% (correlation of 0.88), although it was difficult to verify whether the satellite observations corresponded to bankfull conditions. Bankfull depth estimates were compared with in situ measurements at sites in the Ohio and Willamette rivers, producing a mean error of 24%. 

In [43]:
# note: this takes long to read in
shapefile_dir = "../data/wd_data/asia"
gdf = gpd.read_file(shapefile_dir)

gdf.head()

KeyboardInterrupt: 

In [46]:
gdf.columns

Index(['cat', 'a_cat', 'a_ARCID', 'a_UP_CELLS', 'a_AREA', 'a_WIDTH',
       'a_WIDTH5', 'a_WIDTH95', 'a_DEPTH', 'a_DEPTH5', 'a_DEPTH95', 'b_cat',
       'geometry', 'points', 'keep'],
      dtype='object')

In [47]:
filtered_gdf = gdf[['a_AREA', 'a_WIDTH', 'a_DEPTH', 'geometry']].copy()
filtered_gdf.rename(columns={'a_AREA': 'area', 'a_WIDTH': 'width', 'a_DEPTH': 'depth'}, inplace=True)
filtered_gdf.head()


,area,width,depth,geometry
0,54.92,8.80,0.32,"LINESTRING (93.77500 59.99375, 93.78125 59.99375)"
1,57.18,4.47,0.19,"LINESTRING (99.39167 60.00000, 99.39792 59.993..."
2,35.49,7.08,0.27,"LINESTRING (114.46250 59.99167, 114.45625 59.9..."
3,2179.32,75.47,1.69,"LINESTRING (161.53333 59.99375, 161.55208 59.9..."
4,442.76,25.02,0.71,"LINESTRING (91.80208 59.99375, 91.80625 59.98958)"


In [51]:
# get midpoints of geometries column LINESTRINGS to use for matching 
# also takes long to run

filtered_gdf['points'] = None
for index, row in filtered_gdf.iterrows():
    # Compute the midpoint of the LINESTRING
    geometry = row['geometry']
    midpoint = geometry.interpolate(0.5, normalized=True)
    filtered_gdf.at[index, 'points'] = midpoint

In [52]:
filtered_gdf.shape

(928305, 5)

In [54]:
# Filter gdf to keep only the points in the right area
filtered_gdf['keep'] = filtered_gdf['points'].apply(filter_points)
filtered_gdf = filtered_gdf[filtered_gdf['keep']]
filtered_gdf = filtered_gdf.drop(columns=['keep', 'geometry'])
filtered_gdf = filtered_gdf.rename(columns={'points': 'geometry'})

filtered_gdf.shape

(38064, 4)

## Matching

In [11]:
# read the cleaned data 
dis = pd.read_csv('../clean_data/concatenated_data.csv') # same as overlapping_coords in geo_exploration notebook
dis.head()

,time,lon,lat,dis24,geometry
0,2015-01-01,83.75,29.85,31.251619,POINT (83.75 29.85)
1,2015-01-01,84.05,29.55,40.950394,POINT (84.05000000000001 29.549999999999997)
2,2015-01-01,84.35,29.55,44.086777,POINT (84.35000000000002 29.549999999999997)
3,2015-01-01,84.75,29.25,56.088715,POINT (84.75 29.25)
4,2015-01-01,85.45,29.25,78.481102,POINT (85.45000000000005 29.25)


In [14]:
unique_river_points = dis['geometry'].unique()

In [38]:
def extract_coordinates(coord_string):
    """Extract coordinates from a string in the format 'POINT (x y)'."""
    # Remove the 'POINT' prefix and parentheses, then split on whitespace
    coords = coord_string.replace('POINT', '').replace('(', '').replace(')', '').split()
    # Convert coordinates to floats and return as a tuple
    return float(coords[0]), float(coords[1])


def get_closest_no_time(df, river_points, nn=5):
    """Find the closest point in the df to each point in the river."""
    
    # Create an empty DataFrame to store the closest points
    closest_points = pd.DataFrame(columns=df.columns)

    # Convert river_points from strings to Shapely Point objects
    river_points = [Point(extract_coordinates(x)) for x in river_points]

    tree = BallTree(df['geometry'].apply(lambda x: [x.coords[0][0], x.coords[0][1]]).tolist())
        
    # Find the closest points in one_day_data to each point in river_points
    for true_point in river_points:
        # print(true_point)
        # print(type(true_point))
        _, ind = tree.query([[true_point.x, true_point.y]], k=nn)  # Find NNs --> 1 is too few

        if len(ind) > 0:
            # Choose closest point among 5 NNs
            min_distance = float('inf')
            closest_row = None

            for i in ind[0]:
                distance = calculate_distance(df.iloc[i]['geometry'], true_point)
                if distance < min_distance:
                    min_distance = distance
                    closest_row = df.iloc[i]
            
            closest_points.loc[len(closest_points.index)] = closest_row

    print('Finished finding closest points, converting to geo.')
    
    geo_points = gpd.GeoDataFrame(closest_points, 
                                  crs={'init':'epsg:4326'}, 
                                  geometry=closest_points['geometry'])

    return geo_points

In [39]:
# river points should be a list of tuples of coords to match i.e. from the cleaned data 
matched = get_closest_no_time(filtered_gdf, unique_river_points, 3)

Finished finding closest points, converting to geo.


/Users/smart/Library/Python/3.9/lib/python/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [40]:
matched.head()

,cat,a_cat,a_ARCID,a_UP_CELLS,a_AREA,a_WIDTH,a_WIDTH5,a_WIDTH95,a_DEPTH,a_DEPTH5,a_DEPTH95,b_cat,geometry
0,583268,662408,662408,105984,19681.16,300.57,93.49,979.0,4.96,2.05,12.47,1,POINT (83.75137 29.84792)
1,588351,667655,667655,5015,25278.63,478.6,146.12,1588.17,7.13,2.91,18.09,1,POINT (84.05061 29.54644)
2,588013,667307,667307,141075,26312.56,489.12,149.2,1624.49,7.25,2.96,18.41,1,POINT (84.33542 29.53542)
3,592701,672251,672251,168585,31366.73,538.02,163.49,1793.72,7.81,3.18,19.87,1,POINT (84.73333 29.24583)
4,592451,671983,671983,213187,39707.94,611.42,184.85,2048.88,8.63,3.51,22.01,1,POINT (85.44705 29.26042)


In [41]:
matched.shape

(50, 13)